In [15]:
import fileinput
import re
import sys
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import numpy as np

/home/justm/anaconda3/envs/uw_env/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [1]:
for line in fileinput.input(['train.tsv'], inplace=True, backup='.bak'):
    sys.stdout.write(re.sub(r'(?<!,)"(?!,)',"", line))

data = pd.read_csv('train.tsv', sep='\t', engine='python',  error_bad_lines=False,
                   header=None, index_col=0).dropna()

/home/justm/anaconda3/envs/uw_env/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Skipping line 24551: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 34985: field larger than field limit (131072)
Skipping line 76012: field larger than field limit (131072)


In [2]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.1, random_state=57)

In [3]:
from collections import defaultdict, Counter
a_topic = defaultdict(Counter)
def idf_row(row):
    def prepare_tfidf(text, topic):
        text = re.findall(r"\w+|[^\w\s]", text.lower(), re.UNICODE)
        a_topic[topic] += Counter(text)
    topics = row[3].split(',')
    topics.append('100')
    for topic in topics:
        prepare_tfidf(row[1], topic)
    
data_train.progress_apply(idf_row, axis = 1)

0
28031     None
109442    None
118025    None
111164    None
17914     None
          ... 
85146     None
111105    None
35881     None
12003     None
35826     None
Length: 113335, dtype: object

In [4]:
len(a_topic['100'])

102850

In [5]:
import pickle
with open('counter_title.pickle', 'wb') as f:
    pickle.dump(a_topic, f)

In [6]:
stop_words_90 = []
stop_words_60 = []
stop_words_30 = []
rare_words = []
bad_rare_words = []
for word in tqdm_notebook(a_topic['100'].most_common(102850)):
    tmp = 0
    tmp2 = 0
    for i in range(100):
        tmp1 = a_topic[str(i)].keys()
        if word[0] in tmp1:
            tmp += 1
            tmp2 = max(tmp2, a_topic[str(i)][word[0]])
    if tmp > 90:
        stop_words_90.append(word)
    if tmp > 60:
        stop_words_60.append(word)
    if tmp > 30:
        stop_words_30.append(word)
    if word[1] / tmp >= 3:
        if tmp <= 10:
            rare_words.append(word)
    if tmp2 < 2:
        bad_rare_words.append(word)

In [7]:
rare_answers = defaultdict(list)
for word in rare_words:
    for i in range(100):
        if (word[0] in a_topic[str(i)].keys()) and (a_topic[str(i)][word[0]]>=3):
            rare_answers[word].append(i)

In [16]:
sum_theme = [sum(a_topic[str(i)].values()) for i in range(100)]
words = list(a_topic['100'].keys())
tf_arr = np.zeros((100, len(words)))
for i in tqdm_notebook(range(100)):
    for j in range(len(words)):
        if words[j] in a_topic[str(i)]:
            tf_arr[i, j] = a_topic[str(i)][words[j]] / sum_theme[i]

In [17]:
idf_arr = np.zeros(len(words))
for j in range(len(words)):
    idf_arr[j] = np.log(100 / sum([words[j] in a_topic[str(i)] for i in range(100)]))
tfidf = np.array(tf_arr, copy = True)
for k in range(len(words)):
    tfidf[:,k] = tf_arr[:,k]  * idf_arr[k]

In [18]:
words_dict = dict(zip(words, list(range(len(words)))))
words_dict['украины']

1240

In [19]:
tfidf.shape

(100, 102850)

In [9]:
a_topic_text = defaultdict(Counter)
def idf_row(row):
    def prepare_tfidf(text, topic):
        text = re.findall(r"\w+|[^\w\s]", text.lower(), re.UNICODE)
        a_topic_text[topic] += Counter(text)
    topics = row[3].split(',')
    topics.append('100')
    for topic in topics:
        prepare_tfidf(row[2], topic)
    
data_train.progress_apply(idf_row, axis = 1)

0
28031     None
109442    None
118025    None
111164    None
17914     None
          ... 
85146     None
111105    None
35881     None
12003     None
35826     None
Length: 113335, dtype: object

In [10]:
len(a_topic_text['100'])

836577

In [11]:
import pickle
with open('counter_text.pickle', 'wb') as f:
    pickle.dump(a_topic_text, f)

In [12]:
stop_words1_90 = []
stop_words1_70 = []
stop_words1_50 = []
rare_words1 = []
bad_rare_words1 = []
for word in tqdm_notebook(a_topic_text['100'].most_common(882102)):
    tmp = 0
    tmp2 = 0
    for i in range(100):
        tmp1 = a_topic_text[str(i)].keys()
        if word[0] in tmp1:
            tmp += 1
            tmp2 = max(tmp2, a_topic_text[str(i)][word[0]])
    if tmp > 90:
        stop_words1_90.append(word)
    if tmp > 70:
        stop_words1_70.append(word)
    if tmp > 50:
        stop_words1_50.append(word)
    if word[1] // tmp >= 10:
        if tmp <= 10:
            rare_words1.append(word)
    if tmp2 < 5:
        bad_rare_words1.append(word)

In [13]:
rare_answers1 = defaultdict(list)
for word in rare_words1:
    for i in range(100):
        if (word[0] in a_topic_text[str(i)].keys()) and (a_topic_text[str(i)][word[0]]>=3):
            rare_answers1[word].append(i)

In [20]:
sum_theme = [sum(a_topic_text[str(i)].values()) for i in range(100)]
words = list(a_topic_text['100'].keys())
tf_arr = np.zeros((100, len(words)))
for i in tqdm_notebook(range(100)):
    for j in range(len(words)):
        if words[j] in a_topic_text[str(i)]:
            tf_arr[i, j] = a_topic_text[str(i)][words[j]] / sum_theme[i]
idf_arr = np.zeros(len(words))
for j in range(len(words)):
    idf_arr[j] = np.log(100 / sum([words[j] in a_topic_text[str(i)] for i in range(100)]))
tfidf_text = np.array(tf_arr, copy = True)
for k in range(len(words)):
    tfidf_text[:,k] = tf_arr[:,k]  * idf_arr[k]
words_dict_text = dict(zip(words, list(range(len(words)))))
tfidf_text.shape

(100, 836577)

In [177]:
import string
import numpy as np
from numpy import linalg as LA
stop_words3 = set(map(lambda x: x[0], stop_words1_90))
stop_words4 = set(map(lambda x: x[0], bad_rare_words1))
a = []
a_len = []
for i in tqdm_notebook(range(len(data_train[2]))):
    text = data_train[2][i]
    text = re.findall(r"\w+|[^\w\s]", text.lower(), re.UNICODE)
    tmp = []
    j = 0
    for word in text:
        if word not in stop_words3:
            if word not in stop_words4:
                try:
                    if tfidf_text[:,words_dict_text[word]].sum()>0:
                        if j < 240:
                            tmp.append(tfidf_text[:,words_dict_text[word]])# / LA.norm(tfidf_text[:,words_dict_text[word]]))
                            j+=1
                except:
                    pass
    a_len.append(j)
    while j < 240:
        tmp.append(np.zeros(100))
        j+= 1
    a.append(np.array(tmp))
pd.Series(a_len).quantile(0.1), pd.Series(a_len).quantile(0.5), pd.Series(a_len).quantile(0.9), pd.Series(a_len).quantile(0.95)

(21.0, 66.0, 172.0, 240.0)

In [178]:
X_train = np.array(a)
b = []
for i in range(data_train.shape[0]):
    tmp = np.zeros(100)
    for j in data_train[3][i].split(','):
        tmp[int(j)] = 1
    b.append(tmp)
y_train = np.array(b)

In [179]:
import string
import numpy as np
from numpy import linalg as LA
stop_words3 = set(map(lambda x: x[0], stop_words1_90))
stop_words4 = set(map(lambda x: x[0], bad_rare_words1))
a = []
a_len = []
for i in tqdm_notebook(range(len(data_test[2]))):
    text = data_test[2][i]
    text = re.findall(r"\w+|[^\w\s]", text.lower(), re.UNICODE)
    tmp = []
    j = 0
    for word in text:
        if word not in stop_words3:
            if word not in stop_words4:
                try:
                    if tfidf_text[:,words_dict_text[word]].sum()>0:
                        if j < 240:
                            tmp.append(tfidf_text[:,words_dict_text[word]])# / LA.norm(tfidf_text[:,words_dict_text[word]]))
                            j+=1
                except:
                    pass
    a_len.append(j)
    while j < 240:
        tmp.append(np.zeros(100))
        j+= 1
    a.append(np.array(tmp))

In [180]:
X_test = np.array(a)
b = []
for i in range(data_test.shape[0]):
    tmp = np.zeros(100)
    for j in data_test[3][i].split(','):
        tmp[int(j)] = 1
    b.append(tmp)
y_test = np.array(b)

In [181]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
import keras
hidden_dims1 = 200
hidden_dims2 = 100
reg = keras.regularizers.l1_l2(l1=0.0001, l2=0.001)

In [182]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth=True
config.log_device_placement=True
with tf.device('/GPU:0'):
    model = Sequential()
#     model.add(BatchNormalization())
#     model.add(Conv1D(100,
#                      3,
#                      padding='valid',
#                      activation='selu',
#                      strides=1,
#                      kernel_regularizer=reg))
#     model.add(Dropout(0.1))
#     model.add(GlobalAveragePooling1D())

    # model.add(Flatten())
    # We add a vanilla hidden layer:
    model.add(Dense(200, activation='selu', input_shape=(100,), kernel_regularizer=reg))
    model.add(Dropout(0.1))
#     model.add(Dense(200, activation='selu'))
#     model.add(Dropout(0.05))
#     model.add(Dense(150, activation='selu', kernel_regularizer=reg))
#     model.add(Dropout(0.1))
    model.add(Dense(100, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='Nadam',
                  metrics=['categorical_accuracy'])#, 'top_k_categorical_accuracy'])

    # model.summary()

In [183]:
X_test.shape

(12593, 240, 100)

In [184]:
y_test.shape

(12593, 100)

In [185]:
X_test.sum(axis=1).shape

(12593, 100)

In [186]:
history = model.fit(X_train.sum(axis=1), y_train,
                  batch_size=128,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_test.sum(axis=1), y_test))
score = model.evaluate(X_test.sum(axis=1), y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# print('Test top_k_accuracy:', score[2])
history = model.fit(X_train.sum(axis=1), y_train,
                  batch_size=512,
                  epochs=10,
                  verbose=1,
                  validation_data=(X_test.sum(axis=1), y_test))
score = model.evaluate(X_test.sum(axis=1), y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# print('Test top_k_accuracy:', score[2])
history = model.fit(X_train.sum(axis=1), y_train,
                  batch_size=2048,
                  epochs=10,
                  verbose=1,
                  validation_data=(X_test.sum(axis=1), y_test))
score = model.evaluate(X_test.sum(axis=1), y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# print('Test top_k_accuracy:', score[2])
history = model.fit(X_train.sum(axis=1), y_train,
                  batch_size=8196,
                  epochs=5,
                  verbose=1,
                  validation_data=(X_test.sum(axis=1), y_test))
score = model.evaluate(X_test.sum(axis=1), y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# print('Test top_k_accuracy:', score[2])

Train on 113335 samples, validate on 12593 samples
Epoch 1/20
113335/113335 [==============================] - 5s 48us/step - loss: 8.4504 - categorical_accuracy: 0.0709 - val_loss: 8.4801 - val_categorical_accuracy: 0.0741
Epoch 2/20
113335/113335 [==============================] - 5s 45us/step - loss: 8.4307 - categorical_accuracy: 0.0727 - val_loss: 8.4783 - val_categorical_accuracy: 0.0863
Epoch 3/20
113335/113335 [==============================] - 5s 45us/step - loss: 8.4294 - categorical_accuracy: 0.0750 - val_loss: 8.4790 - val_categorical_accuracy: 0.0741
Epoch 4/20
113335/113335 [==============================] - 5s 44us/step - loss: 8.4207 - categorical_accuracy: 0.0847 - val_loss: 8.4604 - val_categorical_accuracy: 0.0741
Epoch 5/20
113335/113335 [==============================] - 5s 45us/step - loss: 8.3812 - categorical_accuracy: 0.1042 - val_loss: 8.3974 - val_categorical_accuracy: 0.1248
Epoch 6/20
113335/113335 [==============================] - 5s 45us/step - loss: 8.3

In [161]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import gc
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.9
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
reset_keras()

Exception ignored in: <function BaseSession._Callable.__del__ at 0x7f8de8de2b00>
Traceback (most recent call last):
  File "/home/justm/anaconda3/envs/uw_env/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')
Exception ignored in: <function BaseSession._Callable.__del__ at 0x7f8de8de2b00>
Traceback (most recent call last):
  File "/home/justm/anaconda3/envs/uw_env/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


1297258
